## Создаём предметно-ориентированного чат-бота

Для создания чат-бота, который может отвечать на вопросы по какой-то конкретной теме или предметной области, используется подход Retrieval-Augmented Generation (RAG).

Для начала установим необходимые библиотеки. Мы будем использовать библиотеку LangChain, поскольку она содержит в себе все необходимые компоненты для создания таких чат-ботов:
* общение с языковыми моделями
* векторная база данных
* эмбеддинги

In [1]:
%pip install langchain langchain_community yandexcloud gigachat chromadb langchain_chroma huggingface_hub sentence_transformers telebot

Мы будем строить бота-консультанта для Школы дизайна ВШЭ. Для построения бота нужна текстовая база знаний, содержащая всю необходимую информацию. В идеале она должны быть разбита не небольшие кусочки, каждый из которых содержит законченный фрагмент информации.

Возьмём текст с сайта ШД, немного причёсанный и разбитый на фрагменты символами `//`:

In [2]:
!wget https://raw.githubusercontent.com/lanapovsonic/openback/e821e76c549dcd7caff77ee99ceab21835bdedd1/openback

--2026-02-21 16:55:22--  https://raw.githubusercontent.com/lanapovsonic/openback/e821e76c549dcd7caff77ee99ceab21835bdedd1/openback
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7151 (7.0K) [text/plain]
Saving to: ‘openback.2’

openback.2          100%[===================>]   6.98K  --.-KB/s    in 0s      

2026-02-21 16:55:22 (94.4 MB/s) - ‘openback.2’ saved [7151/7151]



In [3]:
!head openbac.txt

Онлайн-бакалавриат Школы дизайна НИУ ВШЭ — совместный проект Школы дизайна НИУ ВШЭ и НИУ ВШЭ — Пермь. Это инновационная программа, позволяющая объединить преимущества проектного подхода к дизайн-образованию и дистанционного формата. По окончании онлайн-бакалавриата Школы дизайна студенты получают диплом о высшем образовании очной формы обучения.
\\
Онлайн-бакалавриат Школы дизайна НИУ ВШЭ готовит востребованных профессионалов, которые могут работать как в офисе, так и удалённо, как в российских, так и в международных компаниях, агентствах, проектах. В рамках программы в настоящий момент открыто семь профилей обучения: «Коммуникационный дизайн», «Анимация и иллюстрация», «Гейм-дизайн», «Дизайн и программирование», «Дизайн и продвижение цифрового продукта», «Дизайн среды и интерьера», «3D-постпродакшн».
\\
О ПРОГРАММЕ
Обучение в онлайн-бакалавриате Школы дизайна построено на принципах проектного подхода: с первого занятия каждый студент делает собственный проект, получая навыки работы с 

В качестве языковой модели будем использовать Gigachat, поэтому нам потребуется ключ, который можно получить [по инструкции](https://developers.sber.ru/docs/ru/gigachat/quickstart/ind-using-api).

Импортируем необходимые библиотеки и создадим объекты:
* GPT для вызова Gigachat
* embeddings для вычисления семантических эмбеддингов текста. Для этого будем использовать мультиязычную модель [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large).

> Для вычисления эмбеддингов было бы идеально тоже использовать облачный сервис от Gigachat или Yandex, но Gigachat требует оплату за его использование.

> Модель эмбеддингов будет вычисляться внутри Colab, поэтому если не включить GPU - на индексирование всего текста может потребоваться 2-3 минуты.

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models.gigachat import GigaChat
from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

gigachat_creds = userdata.get('gigachat_creds')

GPT = GigaChat(
    credentials=gigachat_creds,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    streaming=False,
    verify_ssl_certs=False,
)

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large"
)


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: intfloat/multilingual-e5-large
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузим документ и разобьем его на фрагменты:

In [5]:
doc = open('openbac.txt',encoding='utf-8').read()
docs = doc.split('\\')
print(f"Всего фрагментов: {len(docs)}")
print(f"Макс длина фрагмента: {max(map(len,docs))}")

Всего фрагментов: 63
Макс длина фрагмента: 1314


Поскольку длина макс фрагмента не превышает 1500 символов (500 токенов), то эти фрагменты можно дополнительно не разбивать на части. С учётом такой длины мы можем добавлять в запрос 3-5 найденных фрагментов текста.

Посмотрим, как работает вычисление эмбеддингов:

In [6]:
res = embeddings.embed_documents(docs[0])
print(f"Длина эмбеддингов: {len(res[0])}")

Длина эмбеддингов: 1024


Для хранения всех документов, проиндексированных по эмбеддингам, будем использовать вектоную базу данных, например, ChromaDB. При добавлении текстов в эту базу они будут автоматически проиндексированы:

In [7]:
db = Chroma('vecstore',embeddings,'./db')
db.add_texts(docs)

['1acbd6e5-594b-41c7-94f5-1999bc8496a4',
 '4f00e013-0601-4be5-97e2-ae257f1a464b',
 'a287a186-5438-4b0d-b260-ea809fb12140',
 '0e14f441-e898-41b5-a63b-46b3487c5901',
 '15d4d6dd-82ba-4f4d-87ba-9b568424f04b',
 '12eeb1e0-23cd-4f7a-8537-feb531fdaca0',
 '5a9df612-245c-4446-a39c-83025647ffce',
 '6b44c2ea-6a5f-495b-9315-22f80d14e422',
 'd06c081d-0a1e-4ab2-b140-685e08f4c071',
 'f22c3d14-278c-493f-b8d2-0d67a20a7f9e',
 '3f2f02c1-6786-4e83-8ac5-64b2f05fa36c',
 'cbadd281-e6ee-456f-af36-d645b2df2b6f',
 '5812327f-1db1-44d2-8efa-ba320a6e4eeb',
 '6286a04a-d166-4541-8c65-3758cd96e444',
 '716df069-967f-4b78-a949-8d8f4cd8f694',
 '78e42083-b9bf-4063-b1ce-ecf86fbaecb2',
 '65acb157-0cb2-497a-aa86-f29baa1c5756',
 '9c39b12e-4bf4-48aa-87f5-eecc45bebc1e',
 'cb97e008-1797-4e1b-bab6-7482c428cb8b',
 'ece38ac1-eeea-4b43-b441-c16169789b6d',
 'eb0fc518-7c1a-4fb6-943c-7a7e1c377a0a',
 '7ed62583-26cb-43f1-8eb3-b8b65b2f64e0',
 '3dccddfa-d045-48b3-8f9e-2de64089d51d',
 '9d430d53-a6be-4994-9eba-62b7768d3b53',
 '31483b3d-11f8-

Предположим, мы хотим найти среди документов ответ на вопрос **Сколько баллов ЕГЭ нужно для поступления в школу дизайна?**. Для этого определим объект `retriever`, и передадим ему параметр - количество документов, которые надо найти. После этого поиск сводится к простому вызову `retriever.invoke`

In [8]:
q = "Кто такой продуктовый дизайнер"

retriever = db.as_retriever(search_kwargs={"k": 3 })
retriever.invoke(q)


[Document(id='8770ea3b-2267-41ce-aaa6-8cfcfa80c2e5', metadata={}, page_content='\nБудущие профессии по профилю Дизайн и продвижение цифрового продукта\nВеб-дизайнер, UX/UI-дизайнер, Дизайнер мобильных приложений, Продуктовый дизайнер, Продуктовый аналитик, Арт-директор, Диджитал-маркетолог, Продуктовый маркетолог, Менеджер продукта, Менеджер проекта\n'),
 Document(id='6604adab-069a-4d1a-b2e6-5c485683a4fd', metadata={}, page_content='\nБудущие профессии по профилю Дизайн и продвижение цифрового продукта\nВеб-дизайнер, UX/UI-дизайнер, Дизайнер мобильных приложений, Продуктовый дизайнер, Продуктовый аналитик, Арт-директор, Диджитал-маркетолог, Продуктовый маркетолог, Менеджер продукта, Менеджер проекта\n'),
 Document(id='1b607f55-867f-4e21-b751-97a010cc020f', metadata={}, page_content='\nБудущие профессии по профилю Дизайн и продвижение цифрового продукта\nВеб-дизайнер, UX/UI-дизайнер, Дизайнер мобильных приложений, Продуктовый дизайнер, Продуктовый аналитик, Арт-директор, Диджитал-маркет

Реализуем самую главную функцию ответа на вопрос! Она работает следующим образом:

1. Вызываем `retriever` и получаем 3 релевантных фрагмента текста
2. Объединяем их вместе в контекст `context`
3. Передаём GPT-модели промпт, включающий в себя исходный вопрос и контекст. Модель в этом случае сама смотрит на контекст и находит в нём нужную информацию.

In [9]:
!pip install --upgrade langchain
!pip install chromadb
!pip install --upgrade langchain langchain-community chromadb sentence-transformers
!pip install sentence-transformers

!pip install --upgrade langchain
!pip install chromadb
!pip install langchain-community sentence-transformers

In [10]:
!pip install --upgrade langchain chromadb sentence-transformers

In [19]:
%pip install -U langchain langchain-community chromadb
from langchain_community.vectorstores import Chroma

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

db = Chroma(
    persist_directory="db",
    embedding_function=embeddings
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
def answer(q):
  res = retriever.invoke(q)
  context = '\n'.join(x.page_content for x in res)
  prompt = f"""
  Прочитай следующий текст и используй его при ответе на вопрос далее.
  Используй только информацию из текста. Если в тексте нет ответа на данный вопрос,
  напиши, что не знаешь. Вот текст:\n{context}\nВопрос: {q}"""
  return GPT.invoke(prompt).content

answer(q)

BadRequestError: 400 https://ngw.devices.sberbank.ru:9443/api/v2/oauth: b'{"code":4,"message":"Can\'t decode \'Authorization\' header"}', Headers([('server', 'SynGX'), ('date', 'Sat, 21 Feb 2026 17:08:43 GMT'), ('content-type', 'application/json'), ('content-length', '58'), ('connection', 'keep-alive'), ('vary', 'Origin'), ('vary', 'Access-Control-Request-Method'), ('vary', 'Access-Control-Request-Headers'), ('cache-control', 'no-cache, no-store, max-age=0, must-revalidate'), ('pragma', 'no-cache'), ('expires', '0'), ('x-content-type-options', 'nosniff'), ('strict-transport-security', 'max-age=31536000 ; includeSubDomains'), ('x-frame-options', 'DENY'), ('x-xss-protection', '0'), ('referrer-policy', 'no-referrer'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')])

А теперь оформим это всё в виде телеграм-бота! Для этого используем библиотеку `telebot`. Чтобы создать бота нам нужно сначала получить специальный токен, пообщавшись в telegram со специальным ботом [@botfather](http://t.me/botfather). Полученный токен сохраните в секретах Google Colab.

Код ниже работает в режиме поллинга - он опрашивает сервера telegram на предмет наличия сообщений, и вызывает функцию `handle_message`, если сообщение пришло. Бот в телеграме будет работать только до тех пор, пока следующая ячейка выполняется.

In [14]:
import telebot

telegram_token = userdata.get('tg_token')

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает все про продуктовый дизайн!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ans = answer(message.text)
    bot.send_message(message.chat.id, ans)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе


BadRequestError: 400 https://ngw.devices.sberbank.ru:9443/api/v2/oauth: b'{"code":4,"message":"Can\'t decode \'Authorization\' header"}', Headers([('server', 'SynGX'), ('date', 'Sat, 21 Feb 2026 16:59:59 GMT'), ('content-type', 'application/json'), ('content-length', '58'), ('connection', 'keep-alive'), ('vary', 'Origin'), ('vary', 'Access-Control-Request-Method'), ('vary', 'Access-Control-Request-Headers'), ('cache-control', 'no-cache, no-store, max-age=0, must-revalidate'), ('pragma', 'no-cache'), ('expires', '0'), ('x-content-type-options', 'nosniff'), ('strict-transport-security', 'max-age=31536000 ; includeSubDomains'), ('x-frame-options', 'DENY'), ('x-xss-protection', '0'), ('referrer-policy', 'no-referrer'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')])

### Выводы

С помощью langchain, векторной базы данных и LLM мы создали предметно-ориентированного чатбота. Качество работы такого бота будет зависеть от многих факторов:
* Насколько качественно собрана текстовая база знаний
* Насколько атомарна информация, содержащаяся во фрагментах базы знаний
* Насколько хорошо работает поиск по эмбеддингам (существуют и другие методы поиска, которые также учитывают ключевые слова)
* От качества работы LLM